In [1]:
import pandas as pd

from ccsblib import paros_connection

In [30]:
#qry = """select *
#from tf_validation.validation_cp
#where standard_batch = 'TFv02';"""
qry = """
select a.*, b.source
from tf_validation.validation_cp as a
left join tf_validation.validation_source as b
on a.standard_batch = b.standard_batch
and a.test_orf_ida = b.orf_id2
and a.test_orf_idb = b.orf_id1
where a.standard_batch = 'TFv02';
"""
df = pd.read_sql(qry, paros_connection())
missing = pd.read_excel('TFv02_empty_wells_after_cp.xlsx', sheet_name='Empty destination wells')
missing['Destination_plate'] = missing['Destination_plate'].str.slice(0, -3)
missing['Destination_well'] = missing['Destination_well'].apply(lambda x: x[0] + str(x[1:]).zfill(2))

In [31]:
missing.shape

(47, 5)

In [32]:
df = pd.merge(df, missing, how='inner',
         left_on=['test_pla_full', 'test_pos'],
         right_on=['Destination_plate', 'Destination_well'])
df

,id,test_orf_ida,test_orf_idb,standard_batch,experiment_batch,assay,control,test_pla,test_pos,test_pla_full,...,orf_ida_source_plate,orf_ida_source_well,orf_idb_source_plate,orf_idb_source_well,source,Destination_plate,Destination_well,Destination_well_number,Source_plate,Source_well_number
0,5412,10051,100448,TFv02,None,None,None,1,D09,TFv02N2H_001,...,TFv02_N1N2_Hub1,A01,TFv02R2N2_004_w,F10,whole_tf_genes,TFv02N2H_001,D09,68,TFv02_N1N2_Hub1,1
1,5415,12504,101244,TFv02,None,None,None,1,D12,TFv02N2H_001,...,TFv02R2N1_005_w,H07,TFv02R2N2_004_w,A07,whole_tf_genes,TFv02N2H_001,D12,92,TFv02R2N1_005_w,56
2,5468,71077,101171,TFv02,None,None,None,2,A12,TFv02N2H_002,...,TFv02R2N1_006_w,E08,TFv02_N1N2_Hub1,C01,isoform_matched_negatives,TFv02N2H_002,A12,89,TFv02_N1N2_Hub1,3
3,5480,100016074,101171,TFv02,None,None,None,2,B12,TFv02N2H_002,...,TFv02R2N1_002_w,G10,TFv02_N1N2_Hub1,C01,isoform_matched_negatives,TFv02N2H_002,B12,90,TFv02_N1N2_Hub1,3
4,5532,0,100247,TFv02,None,None,None,2,G04,TFv02N2H_002,...,mN2H_F1_F2,A01,TFv02_N1N2_Hub1,F02,None,TFv02N2H_002,G04,31,TFv02_N1N2_Hub1,14
5,5559,0,101222,TFv02,None,None,None,3,A12,TFv02N2H_003,...,mN2H_F1_F2,A01,TFv02_N1N2_Hub1,H03,None,TFv02N2H_003,A12,89,TFv02_N1N2_Hub1,24
6,5566,10051,100645,TFv02,None,None,None,3,B07,TFv02N2H_003,...,TFv02_N1N2_Hub1,A01,TFv02R2N2_004_w,D07,isoform_positives,TFv02N2H_003,B07,50,TFv02_N1N2_Hub1,1
7,5567,10051,0,TFv02,None,None,None,3,B08,TFv02N2H_003,...,TFv02_N1N2_Hub1,A01,mN2H_F1_F2,F01,None,TFv02N2H_003,B08,58,TFv02_N1N2_Hub1,1
8,5616,11179,100247,TFv02,None,None,None,3,F09,TFv02N2H_003,...,TFv02R2N1_002_w,E04,TFv02_N1N2_Hub1,F02,isoform_matched_negatives,TFv02N2H_003,F09,70,TFv02_N1N2_Hub1,14
9,5618,11179,100178,TFv02,None,None,None,3,F11,TFv02N2H_003,...,TFv02R2N1_002_w,E04,TFv02_N1N2_Hub1,C02,isoform_matched_negatives,TFv02N2H_003,F11,86,TFv02_N1N2_Hub1,11


In [27]:
df['source'].value_counts(dropna=False)

NaN                          16
isoform_matched_negatives    11
isoform_positives             8
isoform_negatives             7
whole_tf_genes                5
Name: source, dtype: int64

In [48]:
df.loc[df['source'].isnull(), ['test_orf_ida', 'test_orf_idb', 'test_pla', 'test_pos']].sort_values(['test_orf_ida', 'test_orf_idb'])

,test_orf_ida,test_orf_idb,test_pla,test_pos
4,0,100247,2,G04
19,0,100313,5,G03
33,0,100313,8,A03
36,0,100455,8,B12
21,0,100565,6,A11
43,0,100565,9,A11
12,0,100568,4,D05
23,0,100703,6,C11
41,0,100703,9,A06
37,0,101070,8,C10


In [37]:
# two questions for the controls
    #1 how many pairs on the plate does it compromise?
    #2 does the control exist on another plate?
qry = """
select a.test_orf_ida, a.test_orf_idb, a.test_pla, a.test_pos, a.test_pla_full, b.source
from tf_validation.validation_cp as a
left join tf_validation.validation_source as b
on a.standard_batch = b.standard_batch
and a.test_orf_ida = b.orf_id2
and a.test_orf_idb = b.orf_id1
where a.standard_batch = 'TFv02';
"""
full = pd.read_sql(qry, paros_connection())

In [38]:
full.head()

,test_orf_ida,test_orf_idb,test_pla,test_pos,test_pla_full,source
0,54731,101256,1,A01,TFv02N2H_001,whole_tf_genes
1,54731,0,1,A02,TFv02N2H_001,None
2,0,101256,1,A03,TFv02N2H_001,None
3,12504,101256,1,A04,TFv02N2H_001,whole_tf_genes
4,12504,0,1,A05,TFv02N2H_001,None


In [41]:
void = []
for _i, row in df.loc[(df['test_orf_ida'] == 0) | (df['test_orf_idb'] == 0), :].iterrows():
    if row['test_orf_ida'] == 0:
        void.append(full.loc[(full['test_orf_idb'] == row['test_orf_idb'])
                              & (full['test_pla'] == row['test_pla'])])
    else:
        void.append(full.loc[(full['test_orf_ida'] == row['test_orf_ida'])
                              & (full['test_pla'] == row['test_pla'])])
void = pd.concat(void)
void

,test_orf_ida,test_orf_idb,test_pla,test_pos,test_pla_full,source
169,120,100247,2,G03,TFv02N2H_002,isoform_matched_negatives
170,0,100247,2,G04,TFv02N2H_002,None
179,53326,100247,2,H02,TFv02N2H_002,isoform_matched_negatives
196,5424,101222,3,A11,TFv02N2H_003,whole_tf_genes
197,0,101222,3,A12,TFv02N2H_003,None
224,100066358,101222,3,D03,TFv02N2H_003,whole_tf_genes
226,1136,101222,3,D05,TFv02N2H_003,whole_tf_genes
227,842,101222,3,D06,TFv02N2H_003,whole_tf_genes
229,1618,101222,3,D08,TFv02N2H_003,whole_tf_genes
204,10051,100645,3,B07,TFv02N2H_003,isoform_positives


In [42]:
# note: could be some overlap with missing pairs
void['source'].value_counts()

isoform_positives            16
isoform_matched_negatives    12
whole_tf_genes               10
isoform_negatives             6
Name: source, dtype: int64

In [44]:
void = []
for _i, row in df.loc[(df['test_orf_ida'] == 0) | (df['test_orf_idb'] == 0), :].iterrows():
    print(((full['test_orf_ida'] == row['test_orf_ida']) &
          (full['test_orf_idb'] == row['test_orf_idb']) &
          (full['test_pla'] != row['test_pla'])).sum())

8
3
2
4
2
4
7
6
5
7
5
3
5
6


To start with we have:

300 isoform positives
294 isoform negatives

we loose 8 positives and 18 negatives

from the controls, 
16 positives, 18 negatives

